In [1]:
from rwa import * # custom module for Response-Weighted Analysis

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import torch.fx

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import math
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

In [16]:
class LocalizationCNN(nn.Module):
    def __init__(self):
        super(LocalizationCNN, self).__init__()
        
        # Define convolutional layers
        self.conv1 = nn.Conv2d(15, 32, kernel_size=7, padding=1)
        self.norm1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=1)
        self.norm2 = nn.BatchNorm2d(64)
        
        # self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        # self.norm3 = nn.BatchNorm2d(128)
        
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        
        # Define regressor
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 6 * 6, 64),  # Adjust the size based on the output from the last pooling layer
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 6)  # Output Layer for x, y coordinates and velocity
        )

    def forward(self, x):
        # Apply convolutional layers with activations, pooling and normalization
        x1 = self.conv1(x)
        x = self.relu(x1)
        x = self.norm1(self.pool(x))
        x2 = self.conv2(x)
        x = self.relu(x2)
        x = self.norm2(self.pool(x))
        # x3 = self.conv3(x)
        # x = self.relu(x3)
        # x = self.norm3(self.pool(x))
        
        # Apply regressor
        x = self.regressor(x)
        
        return x

In [18]:
# Define the network and the trained model
# visual only - no proprioception; agent learns a policy based on visual input only
net = LocalizationCNN()

# load trained model
model_filename = './cnn_training/localization_cnn32.pt'
net.load_state_dict(torch.load(model_filename, map_location=torch.device('cpu')))

<All keys matched successfully>

# Lab Recording Room

In [ ]:
# experiment settings
batch_num = 50
batch_size = 1000
zero_mean = True
layer = 'conv1'